In [1]:
'''
Objective:
•
Analyze consumer and loan attributes to identify patterns indicating the likelihood of default, aiding in loan approval decisions and risk assessment.
•
Utilize multinomial regression to model loan_status as the output variable, helping classify loan applicants into fully paid, current, or charged-off categories.
Constraints:
•
Ensure compliance with regulatory guidelines and ethical considerations in handling consumer financial data during analysis and decision-making processes.
•
Implement robust statistical methodologies and feature selection techniques to accurately capture the driving factors behind loan defaults and mitigate financial risks for the company.
Benefits/Impact:
•
By identifying predictive factors for loan default, the company can optimize loan approval processes, reduce credit losses, and enhance overall portfolio management, leading to improved financial performance and risk mitigation.
•
Multinomial regression provides actionable insights into loan applicant behavior, enabling proactive measures such as adjusting lending criteria, offering personalized loan terms, and implementing targeted risk mitigation strategies to minimize default risks and maximize profitability.
'''


import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from feature_engine.outliers import Winsorizer
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score
import joblib
import pickle
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV

In [2]:

loan = pd.read_csv('loan.csv', encoding = 'latin', low_memory = False)
loan.head(10)

id  member_id  loan_amnt  funded_amnt  funded_amnt_inv        term  \
0  1077501    1296599       5000         5000           4975.0   36 months   
1  1077430    1314167       2500         2500           2500.0   60 months   
2  1077175    1313524       2400         2400           2400.0   36 months   
3  1076863    1277178      10000        10000          10000.0   36 months   
4  1075358    1311748       3000         3000           3000.0   60 months   
5  1075269    1311441       5000         5000           5000.0   36 months   
6  1069639    1304742       7000         7000           7000.0   60 months   
7  1072053    1288686       3000         3000           3000.0   36 months   
8  1071795    1306957       5600         5600           5600.0   60 months   
9  1071570    1306721       5375         5375           5350.0   60 months   

  int_rate  installment grade sub_grade  ... num_tl_90g_dpd_24m  \
0   10.65%       162.87     B        B2  ...                NaN   
1   15.27%        59.83     C        C4  ...                NaN   
2   15.96%        84.33     C        C5  ...                NaN   
3   13.49%       339.31     C        C1  ...                NaN   
4   12.69%        67.79     B        B5  ...                NaN   
5    7.90%       156.46     A        A4  ...                NaN   
6   15.96%       170.08     C        C5  ...                NaN   
7   18.64%       109.43     E        E1  ...                NaN   
8   21.28%       152.39     F        F2  ...                NaN   
9   12.69%       121.45     B        B5  ...                NaN   

  num_tl_op_past_12m pct_tl_nvr_dlq  percent_bc_gt_75 pub_rec_bankruptcies  \
0                NaN            NaN               NaN                  0.0   
1                NaN            NaN               NaN                  0.0   
2                NaN            NaN               NaN                  0.0   
3                NaN            NaN               NaN                  0.0   
4                NaN            NaN               NaN                  0.0   
5                NaN            NaN               NaN                  0.0   
6                NaN            NaN               NaN                  0.0   
7                NaN            NaN               NaN                  0.0   
8                NaN            NaN               NaN                  0.0   
9                NaN            NaN               NaN                  0.0   

  tax_liens tot_hi_cred_lim total_bal_ex_mort total_bc_limit  \
0       0.0             NaN               NaN            NaN   
1       0.0             NaN               NaN            NaN   
2       0.0             NaN               NaN            NaN   
3       0.0             NaN               NaN            NaN   
4       0.0             NaN               NaN            NaN   
5       0.0             NaN               NaN            NaN   
6       0.0             NaN               NaN            NaN   
7       0.0             NaN               NaN            NaN   
8       0.0             NaN               NaN            NaN   
9       0.0             NaN               NaN            NaN   

  total_il_high_credit_limit  
0                        NaN  
1                        NaN  
2                        NaN  
3                        NaN  
4                        NaN  
5                        NaN  
6                        NaN  
7                        NaN  
8                        NaN  
9                        NaN  

[10 rows x 111 columns]

In [3]:
loan.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39717 entries, 0 to 39716
Columns: 111 entries, id to total_il_high_credit_limit
dtypes: float64(74), int64(13), object(24)
memory usage: 33.6+ MB


In [4]:
loan.isna().sum()

id                                0
member_id                         0
loan_amnt                         0
funded_amnt                       0
funded_amnt_inv                   0
                              ...  
tax_liens                        39
tot_hi_cred_lim               39717
total_bal_ex_mort             39717
total_bc_limit                39717
total_il_high_credit_limit    39717
Length: 111, dtype: int64

In [5]:
isna_columns = list([28, 29, 47,50,53,54,55,61, 62, 63,64])
isna_columns1 = list(range(57,78))+ list(range(80, 105))

In [6]:
l1 = loan.drop(loan.columns[isna_columns1], axis = 1)


In [7]:
l1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39717 entries, 0 to 39716
Data columns (total 65 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   id                           39717 non-null  int64  
 1   member_id                    39717 non-null  int64  
 2   loan_amnt                    39717 non-null  int64  
 3   funded_amnt                  39717 non-null  int64  
 4   funded_amnt_inv              39717 non-null  float64
 5   term                         39717 non-null  object 
 6   int_rate                     39717 non-null  object 
 7   installment                  39717 non-null  float64
 8   grade                        39717 non-null  object 
 9   sub_grade                    39717 non-null  object 
 10  emp_title                    37258 non-null  object 
 11  emp_length                   38642 non-null  object 
 12  home_ownership               39717 non-null  object 
 13  annual_inc      

In [8]:
l2 = l1.drop(l1.columns[isna_columns], axis = 1)

In [9]:
l2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39717 entries, 0 to 39716
Data columns (total 54 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   id                          39717 non-null  int64  
 1   member_id                   39717 non-null  int64  
 2   loan_amnt                   39717 non-null  int64  
 3   funded_amnt                 39717 non-null  int64  
 4   funded_amnt_inv             39717 non-null  float64
 5   term                        39717 non-null  object 
 6   int_rate                    39717 non-null  object 
 7   installment                 39717 non-null  float64
 8   grade                       39717 non-null  object 
 9   sub_grade                   39717 non-null  object 
 10  emp_title                   37258 non-null  object 
 11  emp_length                  38642 non-null  object 
 12  home_ownership              39717 non-null  object 
 13  annual_inc                  397

In [10]:
l2.describe()


#auto EDA

id     member_id     loan_amnt   funded_amnt  \
count  3.971700e+04  3.971700e+04  39717.000000  39717.000000   
mean   6.831319e+05  8.504636e+05  11219.443815  10947.713196   
std    2.106941e+05  2.656783e+05   7456.670694   7187.238670   
min    5.473400e+04  7.069900e+04    500.000000    500.000000   
25%    5.162210e+05  6.667800e+05   5500.000000   5400.000000   
50%    6.656650e+05  8.508120e+05  10000.000000   9600.000000   
75%    8.377550e+05  1.047339e+06  15000.000000  15000.000000   
max    1.077501e+06  1.314167e+06  35000.000000  35000.000000   

       funded_amnt_inv   installment    annual_inc           dti  \
count     39717.000000  39717.000000  3.971700e+04  39717.000000   
mean      10397.448868    324.561922  6.896893e+04     13.315130   
std        7128.450439    208.874874  6.379377e+04      6.678594   
min           0.000000     15.690000  4.000000e+03      0.000000   
25%        5000.000000    167.020000  4.040400e+04      8.170000   
50%        8975.000000    280.220000  5.900000e+04     13.400000   
75%       14400.000000    430.780000  8.230000e+04     18.600000   
max       35000.000000   1305.190000  6.000000e+06     29.990000   

        delinq_2yrs  inq_last_6mths  ...    recoveries  \
count  39717.000000    39717.000000  ...  39717.000000   
mean       0.146512        0.869200  ...     95.221624   
std        0.491812        1.070219  ...    688.744771   
min        0.000000        0.000000  ...      0.000000   
25%        0.000000        0.000000  ...      0.000000   
50%        0.000000        1.000000  ...      0.000000   
75%        0.000000        1.000000  ...      0.000000   
max       11.000000        8.000000  ...  29623.350000   

       collection_recovery_fee  last_pymnt_amnt  collections_12_mths_ex_med  \
count             39717.000000     39717.000000                     39661.0   
mean                 12.406112      2678.826162                         0.0   
std                 148.671593      4447.136012                         0.0   
min                   0.000000         0.000000                         0.0   
25%                   0.000000       218.680000                         0.0   
50%                   0.000000       546.140000                         0.0   
75%                   0.000000      3293.160000                         0.0   
max                7002.190000     36115.200000                         0.0   

       policy_code  acc_now_delinq  chargeoff_within_12_mths  delinq_amnt  \
count      39717.0         39717.0                   39661.0      39717.0   
mean           1.0             0.0                       0.0          0.0   
std            0.0             0.0                       0.0          0.0   
min            1.0             0.0                       0.0          0.0   
25%            1.0             0.0                       0.0          0.0   
50%            1.0             0.0                       0.0          0.0   
75%            1.0             0.0                       0.0          0.0   
max            1.0             0.0                       0.0          0.0   

       pub_rec_bankruptcies  tax_liens  
count          39020.000000    39678.0  
mean               0.043260        0.0  
std                0.204324        0.0  
min                0.000000        0.0  
25%                0.000000        0.0  
50%                0.000000        0.0  
75%                0.000000        0.0  
max                2.000000        0.0  

[8 rows x 31 columns]

In [11]:
data = l2.drop(['id','member_id','url','desc','purpose','title','zip_code','addr_state','earliest_cr_line','policy_code','loan_status','int_rate','sub_grade','emp_title','issue_d','pymnt_plan','revol_util','initial_list_status','application_type'], axis = 1)

In [12]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39717 entries, 0 to 39716
Data columns (total 35 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   loan_amnt                   39717 non-null  int64  
 1   funded_amnt                 39717 non-null  int64  
 2   funded_amnt_inv             39717 non-null  float64
 3   term                        39717 non-null  object 
 4   installment                 39717 non-null  float64
 5   grade                       39717 non-null  object 
 6   emp_length                  38642 non-null  object 
 7   home_ownership              39717 non-null  object 
 8   annual_inc                  39717 non-null  float64
 9   verification_status         39717 non-null  object 
 10  dti                         39717 non-null  float64
 11  delinq_2yrs                 39717 non-null  int64  
 12  inq_last_6mths              39717 non-null  int64  
 13  open_acc                    397

In [13]:
import sweetviz
loan_report = sweetviz.analyze([data, 'data'])
loan_report.show_html('loan_report.html')

                                             |                                             | [  0%]   00:00 ->…

Report loan_report.html was generated! NOTEBOOK/COLAB USERS: the web browser MAY not pop up, regardless, the report IS saved in your notebook/colab files.


In [14]:
numeric_features =data.select_dtypes(exclude = ['object']).columns
numeric_features

Index(['loan_amnt', 'funded_amnt', 'funded_amnt_inv', 'installment',
       'annual_inc', 'dti', 'delinq_2yrs', 'inq_last_6mths', 'open_acc',
       'pub_rec', 'revol_bal', 'total_acc', 'out_prncp', 'out_prncp_inv',
       'total_pymnt', 'total_pymnt_inv', 'total_rec_prncp', 'total_rec_int',
       'total_rec_late_fee', 'recoveries', 'collection_recovery_fee',
       'last_pymnt_amnt', 'collections_12_mths_ex_med', 'acc_now_delinq',
       'chargeoff_within_12_mths', 'delinq_amnt', 'pub_rec_bankruptcies',
       'tax_liens'],
      dtype='object')

In [15]:
categorical_features =data.select_dtypes(include = ['object']).columns
categorical_features

Index(['term', 'grade', 'emp_length', 'home_ownership', 'verification_status',
       'last_pymnt_d', 'last_credit_pull_d'],
      dtype='object')

In [16]:
num_pipeline = Pipeline(steps = [('impute', SimpleImputer(strategy = 'mean'))])
scale_pipeline = Pipeline(steps = [('scale', MinMaxScaler())])
num_preprocessor = ColumnTransformer(transformers=[
    ('num', num_pipeline, numeric_features)
])


In [17]:
impute_data = num_preprocessor.fit(data[numeric_features])


In [18]:
impute_df = pd.DataFrame(impute_data.transform(data), columns= list(num_preprocessor.get_feature_names_out()))
impute_df.isna().sum()

num__loan_amnt                     0
num__funded_amnt                   0
num__funded_amnt_inv               0
num__installment                   0
num__annual_inc                    0
num__dti                           0
num__delinq_2yrs                   0
num__inq_last_6mths                0
num__open_acc                      0
num__pub_rec                       0
num__revol_bal                     0
num__total_acc                     0
num__out_prncp                     0
num__out_prncp_inv                 0
num__total_pymnt                   0
num__total_pymnt_inv               0
num__total_rec_prncp               0
num__total_rec_int                 0
num__total_rec_late_fee            0
num__recoveries                    0
num__collection_recovery_fee       0
num__last_pymnt_amnt               0
num__collections_12_mths_ex_med    0
num__acc_now_delinq                0
num__chargeoff_within_12_mths      0
num__delinq_amnt                   0
num__pub_rec_bankruptcies          0
n

In [19]:
scale_preprocessor = ColumnTransformer(transformers=[('scale', scale_pipeline, impute_df.columns)])

In [20]:
scale_data = scale_preprocessor.fit(impute_df)

In [21]:
scale_df = pd.DataFrame(scale_data.transform(impute_df), columns = list(scale_preprocessor.get_feature_names_out()))
scale_df

scale__num__loan_amnt  scale__num__funded_amnt  \
0                   0.130435                 0.130435   
1                   0.057971                 0.057971   
2                   0.055072                 0.055072   
3                   0.275362                 0.275362   
4                   0.072464                 0.072464   
...                      ...                      ...   
39712               0.057971                 0.057971   
39713               0.231884                 0.231884   
39714               0.130435                 0.130435   
39715               0.130435                 0.130435   
39716               0.202899                 0.202899   

       scale__num__funded_amnt_inv  scale__num__installment  \
0                         0.142143                 0.114137   
1                         0.071429                 0.034230   
2                         0.068571                 0.053230   
3                         0.285714                 0.250965   
4                         0.085714                 0.040403   
...                            ...                      ...   
39712                     0.030714                 0.048647   
39713                     0.025000                 0.201388   
39714                     0.037857                 0.109461   
39715                     0.018571                 0.108329   
39716                     0.022857                 0.185917   

       scale__num__annual_inc  scale__num__dti  scale__num__delinq_2yrs  \
0                    0.003336         0.921974                 0.000000   
1                    0.004336         0.033344                 0.000000   
2                    0.001376         0.290764                 0.000000   
3                    0.007538         0.666889                 0.000000   
4                    0.012675         0.598199                 0.000000   
...                       ...              ...                      ...   
39712                0.017678         0.377793                 0.000000   
39713                0.002335         0.213404                 0.090909   
39714                0.016011         0.076692                 0.000000   
39715                0.032688         0.124041                 0.000000   
39716                0.003002         0.476492                 0.090909   

       scale__num__inq_last_6mths  scale__num__open_acc  scale__num__pub_rec  \
0                           0.125              0.023810                  0.0   
1                           0.625              0.023810                  0.0   
2                           0.250              0.000000                  0.0   
3                           0.125              0.190476                  0.0   
4                           0.000              0.309524                  0.0   
...                           ...                   ...                  ...   
39712                       0.000              0.261905                  0.0   
39713                       0.125              0.095238                  0.0   
39714                       0.000              0.214286                  0.0   
39715                       0.000              0.357143                  0.0   
39716                       0.000              0.119048                  0.0   

       ...  scale__num__total_rec_late_fee  scale__num__recoveries  \
0      ...                        0.000000                0.000000   
1      ...                        0.000000                0.003952   
2      ...                        0.000000                0.000000   
3      ...                        0.094173                0.000000   
4      ...                        0.000000                0.000000   
...    ...                             ...                     ...   
39712  ...                        0.000000                0.000000   
39713  ...                        0.000000                0.000000   
39714  ...                        0.000000                0.000000   
397

In [22]:
joblib.dump(impute_data, 'multinomial_assignment_impute')
joblib.dump(scale_data, 'multinomial_assignment_scale')

#outlier analysis

['multinomial_assignment_scale']

In [23]:
scale_df1 = scale_df.drop(['scale__num__delinq_2yrs', 'scale__num__pub_rec', 'scale__num__out_prncp', 'scale__num__out_prncp_inv', 'scale__num__total_rec_late_fee', 'scale__num__recoveries', 'scale__num__collection_recovery_fee', 'scale__num__collections_12_mths_ex_med', 'scale__num__acc_now_delinq', 'scale__num__chargeoff_within_12_mths', 'scale__num__delinq_amnt', 'scale__num__pub_rec_bankruptcies', 'scale__num__tax_liens'], axis = 1)

In [24]:
winsor = Winsorizer(capping_method = 'iqr',
                   tail = 'both',
                   fold = 1.5,
                   variables = list(scale_df1.columns))
outlier_pipeline = Pipeline([('wins', winsor)])

In [25]:
outlier_data = outlier_pipeline.fit_transform(scale_df)

In [26]:
outlier_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39717 entries, 0 to 39716
Data columns (total 28 columns):
 #   Column                                  Non-Null Count  Dtype  
---  ------                                  --------------  -----  
 0   scale__num__loan_amnt                   39717 non-null  float64
 1   scale__num__funded_amnt                 39717 non-null  float64
 2   scale__num__funded_amnt_inv             39717 non-null  float64
 3   scale__num__installment                 39717 non-null  float64
 4   scale__num__annual_inc                  39717 non-null  float64
 5   scale__num__dti                         39717 non-null  float64
 6   scale__num__delinq_2yrs                 39717 non-null  float64
 7   scale__num__inq_last_6mths              39717 non-null  float64
 8   scale__num__open_acc                    39717 non-null  float64
 9   scale__num__pub_rec                     39717 non-null  float64
 10  scale__num__revol_bal                   39717 non-null  fl

In [27]:
#Encoding

In [28]:
from sklearn.preprocessing import OneHotEncoder
encode_pipeline = Pipeline(steps = [('encode', OneHotEncoder(drop = 'first'))])

encode_preprocessor = ColumnTransformer(transformers=[('encode', encode_pipeline, categorical_features)])

In [29]:
encode_data = encode_preprocessor.fit(data)

In [30]:
encode_df = pd.DataFrame((encode_data.transform(data).toarray()), columns = list(encode_preprocessor.get_feature_names_out()))

In [31]:
encode_df

encode__term_ 60 months  encode__grade_B  encode__grade_C  \
0                          0.0              1.0              0.0   
1                          1.0              0.0              1.0   
2                          0.0              0.0              1.0   
3                          0.0              0.0              1.0   
4                          1.0              1.0              0.0   
...                        ...              ...              ...   
39712                      0.0              0.0              0.0   
39713                      0.0              0.0              1.0   
39714                      0.0              0.0              0.0   
39715                      0.0              0.0              0.0   
39716                      0.0              0.0              0.0   

       encode__grade_D  encode__grade_E  encode__grade_F  encode__grade_G  \
0                  0.0              0.0              0.0              0.0   
1                  0.0              0.0              0.0              0.0   
2                  0.0              0.0              0.0              0.0   
3                  0.0              0.0              0.0              0.0   
4                  0.0              0.0              0.0              0.0   
...                ...              ...              ...              ...   
39712              0.0              0.0              0.0              0.0   
39713              0.0              0.0              0.0              0.0   
39714              0.0              0.0              0.0              0.0   
39715              0.0              0.0              0.0              0.0   
39716              0.0              1.0              0.0              0.0   

       encode__emp_length_10+ years  encode__emp_length_2 years  \
0                               1.0                         0.0   
1                               0.0                         0.0   
2                               1.0                         0.0   
3                               1.0                         0.0   
4                               0.0                         0.0   
...                             ...                         ...   
39712                           0.0                         0.0   
39713                           0.0                         0.0   
39714                           0.0                         0.0   
39715                           0.0                         0.0   
39716                           0.0                         0.0   

       encode__emp_length_3 years  ...  encode__last_credit_pull_d_Sep-07  \
0                             0.0  ...                                0.0   
1                             0.0  ...                                0.0   
2                             0.0  ...                                0.0   
3                             0.0  ...                                0.0   
4                             0.0  ...                                0.0   
...                           ...  ...                                ...   
39712                         0.0  ...                                0.0   
39713                         1.0  ...                                0.0   
39714                         0.0  ...                                0.0   
39715                         0.0  ...                                0.0   
39716                         0.0  ...                                0.0   

       encode__last_credit_pull_d_Sep-08  encode__last_credit_pull_d_Sep-09  \
0                                    0.0                                0.0   
1                                    0.0                                0.0   
2                                    0.0                                0.0   
3                                    0.0                                0.0   
4                                    0.0                                0.0   
...                                  ...                     

In [33]:
cleandata = pd.concat([scale_df, encode_df], axis = 1)
cleandata

scale__num__loan_amnt  scale__num__funded_amnt  \
0                   0.130435                 0.130435   
1                   0.057971                 0.057971   
2                   0.055072                 0.055072   
3                   0.275362                 0.275362   
4                   0.072464                 0.072464   
...                      ...                      ...   
39712               0.057971                 0.057971   
39713               0.231884                 0.231884   
39714               0.130435                 0.130435   
39715               0.130435                 0.130435   
39716               0.202899                 0.202899   

       scale__num__funded_amnt_inv  scale__num__installment  \
0                         0.142143                 0.114137   
1                         0.071429                 0.034230   
2                         0.068571                 0.053230   
3                         0.285714                 0.250965   
4                         0.085714                 0.040403   
...                            ...                      ...   
39712                     0.030714                 0.048647   
39713                     0.025000                 0.201388   
39714                     0.037857                 0.109461   
39715                     0.018571                 0.108329   
39716                     0.022857                 0.185917   

       scale__num__annual_inc  scale__num__dti  scale__num__delinq_2yrs  \
0                    0.003336         0.921974                 0.000000   
1                    0.004336         0.033344                 0.000000   
2                    0.001376         0.290764                 0.000000   
3                    0.007538         0.666889                 0.000000   
4                    0.012675         0.598199                 0.000000   
...                       ...              ...                      ...   
39712                0.017678         0.377793                 0.000000   
39713                0.002335         0.213404                 0.090909   
39714                0.016011         0.076692                 0.000000   
39715                0.032688         0.124041                 0.000000   
39716                0.003002         0.476492                 0.090909   

       scale__num__inq_last_6mths  scale__num__open_acc  scale__num__pub_rec  \
0                           0.125              0.023810                  0.0   
1                           0.625              0.023810                  0.0   
2                           0.250              0.000000                  0.0   
3                           0.125              0.190476                  0.0   
4                           0.000              0.309524                  0.0   
...                           ...                   ...                  ...   
39712                       0.000              0.261905                  0.0   
39713                       0.125              0.095238                  0.0   
39714                       0.000              0.214286                  0.0   
39715                       0.000              0.357143                  0.0   
39716                       0.000              0.119048                  0.0   

       ...  encode__last_credit_pull_d_Sep-07  \
0      ...                                0.0   
1      ...                                0.0   
2      ...                                0.0   
3      ...                                0.0   
4      ...                                0.0   
...    ...                                ...   
39712  ...                                0.0   
39713  ...                                0.0   
39714  ...                                0.0   
39715  ...                                0.0   
39716  ...                                0.0   

       encode__last_credit_pull_d_Sep-08  encode__last_credit_pull_d_Sep-09  \
0                                  

In [34]:
joblib.dump(impute_data, 'multinomial_assignment_impute')
joblib.dump(outlier_data, 'multinomial_assignment_outlier')
joblib.dump(scale_data, 'multinomial_assignment_scale')
joblib.dump(encode_data, 'multinomial_assignment_encode')

['multinomial_assignment_encode']

In [52]:
from sklearn.preprocessing import LabelEncoder
y = pd.DataFrame(l2['loan_status'], columns = ['loan_status'])
label_encoder = LabelEncoder()
y_encode = label_encoder.fit_transform(y)

C:\ProgramData\anaconda3\Lib\site-packages\sklearn\preprocessing\_label.py:114: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [53]:
y_encode

array([2, 0, 2, ..., 2, 2, 2])

In [43]:
y.columns = y_data.get_feature_names_out()
y

loan_status_Charged Off  loan_status_Current  loan_status_Fully Paid
0                          0.0                  0.0                     1.0
1                          1.0                  0.0                     0.0
2                          0.0                  0.0                     1.0
3                          0.0                  0.0                     1.0
4                          0.0                  1.0                     0.0
...                        ...                  ...                     ...
39712                      0.0                  0.0                     1.0
39713                      0.0                  0.0                     1.0
39714                      0.0                  0.0                     1.0
39715                      0.0                  0.0                     1.0
39716                      0.0                  0.0                     1.0

[39717 rows x 3 columns]

In [44]:
#data segmendation for training and testing

In [54]:
x_train, x_test, y_train, y_test = train_test_split(cleandata,y_encode, test_size = 0.2, random_state = 42)

In [46]:
#model building

In [55]:
logmodel = LogisticRegression(multi_class = 'multinomial', solver = 'lbfgs', max_iter = 5)

In [56]:
model = logmodel.fit(x_train, y_train)

C:\ProgramData\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [57]:
train_pred = model.predict(x_train)
accuracy_score(y_train, train_pred)

0.8747049381550372

In [58]:
#test accuracy
test_pred = model.predict(x_test)
accuracy_score(y_test, test_pred)

0.8790281973816717

In [59]:
# Hyperparameter Optimization

logmodel1 = LogisticRegression(multi_class = 'multinomial')

In [ ]:
param_grid = [
    {'penalty': ['l1','l2', 'elasticnet', 'none'], 'C': np.logspace(-4, 4, 20), 'solver': ['lbfgs', 'newton-cg','liblinear','sag','saga'], 'max_iter': [3,4,5,6]}
]

In [ ]:
clf = GridSearchCV(logmodel1, param_grid= param_grid, cv = 5, verbose = True, n_jobs = -1)

In [ ]:
best_clf = clf.fit(x_train, y_train)

In [ ]:
best_clf.best_estimator_

In [ ]:
#training accuracy
accuracy_score(y_train, best_clf.predict(x_train))

In [ ]:
#testing accuracy
accuracy_score(y_test, best_clf.predict(x_test))

In [ ]:
pickle.dump(best_clf, open('multinomial_assignment_grid_model.pkl','wb'))